In [1]:
import pandas as pd

In [78]:
# 1. Load CSV files
orders = pd.read_csv("data\\olist_orders_dataset.csv")
items = pd.read_csv("data\\olist_order_items_dataset.csv")
payments = pd.read_csv("data\\olist_order_payments_dataset.csv")
customers = pd.read_csv("data\\olist_customers_dataset.csv")
products = pd.read_csv("data\\olist_products_dataset.csv")
sellers = pd.read_csv("data\\olist_sellers_dataset.csv")
reviews = pd.read_csv("data\\olist_order_reviews_dataset.csv")

# 2. Merge: Orders + Customers
master = pd.merge(orders, customers, on="customer_id", how="left")

# 3. Merge: Add order_items
master = pd.merge(master, items, on="order_id", how="left")

# 4. Merge: Add payments
master = pd.merge(master, payments, on="order_id", how="left")

# 5. Merge: Add products
master = pd.merge(master, products, on="product_id", how="left")

# 6. Merge: Add sellers
master = pd.merge(master, sellers, on="seller_id", how="left")

# 7. Merge: Add reviews
master = pd.merge(master, reviews, on="order_id", how="left")


In [79]:
#creating a new clean df with required cols
columns_needed = [
    'order_id','order_purchase_timestamp','price','freight_value',
    'payment_value','payment_type','customer_state','seller_state',
    'product_category_name','product_weight_g','product_length_cm',
    'product_height_cm','product_width_cm','review_score'
]

df_clean = master[columns_needed].copy()

In [80]:
# filled nulls in height,weight,length cols
df_clean['product_height_cm'] = df_clean['product_height_cm'].fillna(df_clean['product_height_cm'].mean())
df_clean['product_width_cm'] = df_clean['product_width_cm'].fillna(df_clean['product_width_cm'].mean())
df_clean['product_length_cm'] = df_clean['product_length_cm'].fillna(df_clean['product_length_cm'].mean())
# created col volume
df_clean['product_volume']=df_clean['product_height_cm']*df_clean['product_width_cm']*df_clean['product_length_cm']


In [81]:
#dropped height,weight,length cols
df_clean=df_clean.drop('product_height_cm',axis=1)
df_clean=df_clean.drop('product_width_cm',axis=1)
df_clean=df_clean.drop('product_length_cm',axis=1)

In [82]:
# filled nulls in weigth
df_clean['product_weight_g']=df_clean['product_weight_g'].fillna(df_clean['product_weight_g'].mean())

In [85]:
# cfilling nulls
df_clean['price']=df_clean['price'].fillna(df_clean['price'].mean())
df_clean['payment_value']=df_clean['payment_value'].fillna(df_clean['payment_value'].mean())
df_clean['freight_value']=df_clean['freight_value'].fillna(df_clean['freight_value'].mean())
df_clean['review_score']=df_clean['review_score'].fillna(df_clean['review_score'].mean())
df_clean['product_category_name']=df_clean['product_category_name'].fillna("Others")
df_clean['seller_state']=df_clean['seller_state'].fillna("Others")

In [87]:
state_map = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins',
    'Others': 'Others'  #  filled values
}

In [88]:
#df_clean['seller_state']=df_clean['seller_state'].map(state_map)
df_clean['customer_state']=df_clean['customer_state'].map(state_map)

In [90]:
df_clean['seller_state']=df_clean['seller_state'].map(state_map)

In [92]:
df_clean['review_score']=df_clean['review_score'].astype(int)

In [ ]:
# created review review_category
def categorize(scr):
    if scr in [0,1,2]:
        return "Bad"
    elif scr==3:
        return "Average"
    else:
        return "Good"

df_clean['review_category']=df_clean['review_score'].apply(categorize)

In [96]:
df_clean.to_csv('cleaned.csv',index=False)